In [ ]:
import pandas as pd
from datetime import datetime as dt
import datetime as dtp
import numpy as np
from dateutil.relativedelta import relativedelta   
import math
import random as rd

In [ ]:
#create MEV data arrays 
CPIdata = pd.read_csv('MEV\CPI.csv', delimiter = ',',parse_dates=['DATE'],index_col=['DATE']) #monthlydata
CPIdata.columns = ["CPI"]
HPIdata = pd.read_csv('MEV\HPI.csv', delimiter = ',',parse_dates=['DATE'],index_col=['DATE']) # quarterly
HPIdata.columns = ["HPI"]
HPIdata= HPIdata.resample('M').ffill()
PIdata = pd.read_csv('MEV\PINC.csv', delimiter = ',',parse_dates=['DATE'],index_col=['DATE'])# quarterly
PIdata.columns = ["PI"]
PIdata=  PIdata.resample('M').ffill()
RGDPdata = pd.read_csv('MEV\RGDP.csv', delimiter = ',',parse_dates=['DATE'],index_col=['DATE'])# quarterly
RGDPdata.columns = ["RGDP"]
RGDPdata = RGDPdata.resample('M').ffill()
UEMPdata = pd.read_csv('MEV\\UEMP.csv', delimiter = ',',parse_dates=['DATE'],index_col=['DATE'])# quarterly
UEMPdata.columns = ["UEMP"]
UEMPdata = UEMPdata.resample('M').ffill()

#normalize date index 
CPIdata.index = CPIdata.index.strftime('%m%Y')
HPIdata.index = HPIdata.index.strftime('%m%Y')
PIdata.index = PIdata.index.strftime('%m%Y')
RGDPdata.index = RGDPdata.index.strftime('%m%Y')
UEMPdata.index = UEMPdata.index.strftime('%m%Y')
#Create Single Dataframe for MEV data
MEV = pd.concat([CPIdata,HPIdata,RGDPdata,PIdata,UEMPdata],axis=1)

In [ ]:
#Dataset reduction :Raw to reduced data
#Read in column names and the selected columns : driven by CSV inputs
col = pd.read_csv('ColumnsNames.csv', delimiter = ',')
sel_col = pd.read_csv('SelectedColumns.csv')
dataset = pd.DataFrame()
flag_read = 0
filename_loc = ["INPUTS\\2008Q4.csv","INPUTS\\2008Q3.csv"]
seller = ['Bank Of America, N.A.']
prop = ['CA']
# ,"INPUTS\\2007Q2.csv","INPUTS\\2007Q3.csv","INPUTS\\2007Q4.csv","INPUTS\\2006Q1.csv","INPUTS\\2006Q2.csv","INPUTS\\2006Q3.csv","INPUTS\\2006Q4.csv",]
#needs to be in loop reading chunks of data and creating hte final data set. 
for location in filename_loc:
    for chunk in pd.read_csv(location,chunksize=100000,index_col=False, header=None, delimiter="|"):
        chunk.columns = col['Name']
        data = chunk[list(sel_col['Final '])]
        # can filter data here : use california 
        fdata = data.query('`Property State` == @prop')
        fdata = fdata.query('`Seller Name` == @seller')
        # take a subset of the loan ids in the random sample 
        if flag_read == 0:
            dataset=fdata
            #flag_read = 1
        else:
            dataset= dataset.append(fdata)
        flag_read = flag_read + 1
        # if flag_read == 10:# only for testing, will remove for actual data
        #     break
    dataset.to_csv('RedData\RD_' + location[7:])
# dataset.to_csv("Final_Dataset.csv")
# convert to nuemric based on inputs in CSV
#num_cols = list(sel_col[sel_col['Numeric']==1]['Final '])
#dataset[num_cols].apply(pd.to_numeric,errors = 'coerce')



C:\Users\sthakur6\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3364: DtypeWarning: Columns (40,101,105) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\sthakur6\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3364: DtypeWarning: Columns (105) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\sthakur6\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3364: DtypeWarning: Columns (40,105) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\sthakur6\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3364: DtypeWarning: Columns (40,101) have mixed types.Specify dtype option on import or set low_memory=False.
  if (

In [ ]:
file_list = ['ReducedData/RD_2008Q2.csv','ReducedData/RD_2008Q3.csv','ReducedData/RD_2008Q4.csv','ReducedData/RD_2006Q1.csv','ReducedData/RD_2006Q2.csv']
# 'ReducedData/RD_2006Q3.csv','ReducedData/RD_2006Q4.csv','ReducedData/RD_2007Q1.csv','ReducedData/RD_2007Q2.csv','ReducedData/RD_2007Q3.csv','ReducedData/RD_2007Q4.csv','ReducedData/RD_2008Q1.csv','ReducedData/RD_2008Q2.csv','ReducedData/RD_2008Q3.csv','ReducedData/RD_2008Q4.csv','ReducedData/RD_2006Q1.csv','ReducedData/RD_2006Q2.csv']
dataset_final = pd.DataFrame()
for fl in file_list:
    print(fl)
    t = pd.read_csv(fl)
    t= t.drop('Unnamed: 0',axis = 1)
    # print('Debug')

    # print(t['Current Loan Delinquency Status'].value_counts())
    # take random sample of 40% loan ids
    l = t['Loan Identifier'].unique()
    smpl = rd.sample(list(l),int(0.6*len(l)))
    #t['Sample'] = [1 if i in smpl else np.nan for i in t['Loan Identifier'] ]
    t = t.query("`Loan Identifier` == @smpl")
    #t = t.dropna(subset=['Sample'])

    #debug
    print("after sampling")
    # print(t.shape)

    # Add status to data . OPtimize using dictionaries or apply using function
    t['Status'] = [0] *len(t)
    def_stat = {}
    # print(t['Current Loan Delinquency Status'].value_counts())
    t.loc[(t['Current Loan Delinquency Status']=='XX') ,'Zero Balance Code'] = 1
    t.loc[(t['Current Loan Delinquency Status']=='XX') ,'Current Loan Delinquency Status'] = -1
    t.loc[(t['Current Loan Delinquency Status'].astype('int')>=3) & (t['Current Loan Delinquency Status'].astype('int')<9),'Status'] = 1
    t.loc[(t['Current Loan Delinquency Status'].astype('int')>=0) & (t['Current Loan Delinquency Status'].astype('int')<3),'Status'] = 0
    t.loc[(t['Current Loan Delinquency Status'].astype('int')>=9) ,'Status'] = 2
    t.loc[(t['Zero Balance Code'] == 1) ,'Status'] = -1
    # print(t['Status'].value_counts())

    # create target status 
    tar = list(t['Status'][1:len(t)])
    tar.append(0)
    t['Target'] = tar

    # drop rows after default/prepayment : Ananya
    remove_list = []
    templ = t.iloc[0,0]
    flag = 0
    for r_index,i in t.iterrows():
        if (i['Loan Identifier'] == templ):
            if flag == 0:
                if (i['Status'] == 2) or (i['Status']==-1):
                    flag = 1
                    remove_list.append(r_index)
                else:
                    continue
            else:
                remove_list.append(r_index)
        else:
            templ = i['Loan Identifier']
            if (i['Status'] == 2) or (i['Status']==-1):
                flag = 1
            else:
                flag = 0

    t.drop(remove_list,axis= 0,inplace= True )

    #debug
    print("after drop")
    # print(t.shape)

    # Upfill/downfill the missing data in some columns : JACK  
    t.fillna(method = 'ffill', inplace = True)


    # Add MEV to dataset: SID
    #update format for monthly reporting period 
    t['Monthly Reporting Period'] = [i.strftime("%m%Y") for i in pd.to_datetime(t['Monthly Reporting Period'],format = '%m%Y')]
    t['HPI']= [MEV.loc[i,'HPI'] for i in t['Monthly Reporting Period']]
    t['CPI']= [MEV.loc[i,'CPI'] for i in t['Monthly Reporting Period']]
    t['PI']= [MEV.loc[i,'PI'] for i in t['Monthly Reporting Period']]
    t['RGDP']= [MEV.loc[i,'RGDP'] for i in t['Monthly Reporting Period']]
    t['UEMP']= [MEV.loc[i,'UEMP'] for i in t['Monthly Reporting Period']]

    # output to file 
    t.to_csv("FinalData\FD_" +fl[-10:])
    dataset_final = dataset_final.append(t)
dataset_final.to_csv("FinalData.csv")

ReducedData/RD_2008Q2.csv
after sampling
after drop
ReducedData/RD_2008Q3.csv
after sampling
after drop
ReducedData/RD_2008Q4.csv
after sampling
after drop
ReducedData/RD_2006Q1.csv
after sampling
after drop
ReducedData/RD_2006Q2.csv
after sampling
after drop
